In [ ]:
# 1) リポジトリを取ってワークディレクトリへ
%cd /content
!git clone https://github.com/Kohei-Arita/RSNA-2025.git
%cd RSNA-2025

# 念のため存在確認
!ls -la env | head -n 20


In [ ]:
# Colab: ランタイム起動直後に一度だけ
!pip -q install -U pip
!pip -q install -r env/requirements.txt "dvc[gcs]"


In [ ]:
from google.colab import auth
auth.authenticate_user()  # ブラウザで認可
print("✅ ADC 認証OK")


In [ ]:
from google.cloud import storage
client = storage.Client()  # ADC を使う
bucket_name = "<YOUR_BUCKET_NAME>"  # 例: rsna2025-<org>-prod
bucket = client.bucket(bucket_name)
print("✅ バケット存在:", bucket.exists())


In [ ]:
# 既に dvc.config.example がある前提。自分の環境ファイル dvc.config を作るならここで。

# 例: リモートは README 記載のとおり gs://.../rsna2025 を既定に
!cp -n dvc.config.example dvc.config || true
!dvc remote list
!dvc pull -q || echo "（初回は DVC で管理されているファイルが無い/権限不足の場合があります）"
print("✅ DVC pull 実行完了（メッセージを確認）")


In [ ]:
# 追加: GCS 権限チェック（ADC 認証での書き込み/読み取り/削除）
# - ここでは `bucket_name` と `rsna2025` プレフィックス配下に小さなテストオブジェクトを書き込み/読み取り/削除します。
# - すべて成功すれば、少なくとも Object Writer/Delete 相当の権限があることが確認できます。

from datetime import datetime
import uuid
from google.cloud import storage

client = storage.Client()  # ADC を利用
bucket = client.bucket(bucket_name)

prefix_root = "rsna2025"  # DVC リモートのルートに合わせる
blob_path = f"{prefix_root}/_perm_check/colab_{datetime.utcnow().strftime('%Y%m%dT%H%M%SZ')}_{uuid.uuid4().hex[:8]}.txt"
blob = bucket.blob(blob_path)

try:
    blob.upload_from_string(b"perm_check", content_type="text/plain")
    print("✅ GCS 書き込みOK:", f"gs://{bucket_name}/{blob_path}")
    text = blob.download_as_text()
    print("✅ 読み取りOK:", text)
    blob.delete()
    print("✅ 削除OK（Object Admin/Writer 相当の権限あり）")
except Exception as e:
    print("❌ GCS 書き込み/読み取り/削除に失敗:", e)

